import section

In [8]:
from pandas import read_csv # read CSV (Comma- Separated Values ) files into a DataFrame
import numpy as np # numerical computations,working with arrays,mathematical operations,random number generation
from pandas import read_csv
import numpy as np
import tensorflow as tf # Import TensorFlow



K = tf.keras.backend

from keras import Model
from keras.layers import Layer
from keras.layers import Input,Dense,SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error
from keras.models import Sequential # model in keras is a straightforward way to build a neural network layer by layer .it's ideal ofr models where each layer has exactly one input tensor and one output tensor
from keras.metrics import mean_squared_error # import from keras.losses for use as a loss function

Preparing the Dataset


In [9]:
def get_fib_seq(n,scale_data=True):
  #Get the Fibonacci sequence
  seq=np.zeros(n)
  fib_n1=0.0
  fib_n=1.0
  for i in range(n):
    seq[i]=fib_n1+fib_n
    fib_n1=fib_n
    fib_n=seq[i]
  scaler=[]
  if scale_data:
    scaler=MinMaxScaler(feature_range=(0,1))
    seq=np.reshape(seq,(n,1))
    seq=scaler.fit_transform(seq).flatten()
  return seq,scaler

fib_seq=get_fib_seq(10,False)[0]
print(fib_seq)

[ 1.  2.  3.  5.  8. 13. 21. 34. 55. 89.]


In [10]:
def get_fib_XY(total_fib_numbers,time_steps,train_percent,scale_data=True):
  dat,scaler=get_fib_seq(total_fib_numbers,scale_data)
  Y_ind=np.arange(time_steps,len(dat),1)
  Y=dat[Y_ind]
  rows_x=len(Y)
  X=dat[0:rows_x]
  for i in range(time_steps-1):
    temp=dat[i+1:rows_x+i+1]
    X=np.column_stack((X,temp))

  # random permutation with fixed seed
  rand=np.random.RandomState(seed=13)
  idx=rand.permutation(rows_x)
  split=int(train_percent*rows_x)
  train_ind=idx[0:split]
  test_ind=idx[split:]
  trainX=X[train_ind]
  trainY=Y[train_ind]
  testX=X[test_ind]
  testY=Y[test_ind]
  trainX=np.reshape(trainX,(len(trainX),time_steps,1))
  testX=np.reshape(testX,(len(testX),time_steps,1))
  return trainX,trainY,testX,testY,scaler

trainX,trainY,testX,testY,scaler=get_fib_XY(12,3,0.7,False)
print('trainX=',trainX)
print('trainY = ',trainY)



trainX= [[[ 8.]
  [13.]
  [21.]]

 [[ 5.]
  [ 8.]
  [13.]]

 [[ 2.]
  [ 3.]
  [ 5.]]

 [[13.]
  [21.]
  [34.]]

 [[21.]
  [34.]
  [55.]]

 [[34.]
  [55.]
  [89.]]]
trainY =  [ 34.  21.   8.  55.  89. 144.]


Setting Up the network

In [11]:
#Set up parameters
time_steps=20
hidden_units=2
epochs=30

#Create a traditional RNN network
def create_RNN(hidden_units,dense_units,input_shape,activation):
  model=Sequential()
  model.add(SimpleRNN(hidden_units,input_shape=input_shape,activation=activation[0]))
  model.add(Dense(units=dense_units,activation=activation[1]))
  model.compile(loss='mse',optimizer='adam')
  return model

model_RNN=create_RNN(hidden_units=hidden_units,dense_units=1,input_shape=(time_steps,1),activation=['tanh','tanh'])

model_RNN.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 2)                   │               8 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11 (44.00 B)

 Trainable params: 11 (44.00 B)

 Non-trainable params: 0 (0.00 B)

Train the Network And Evaluate

In [13]:
#Generate the dataset
trainX,trainY,testX,testY,scaler=get_fib_XY(1200,time_steps,0.7)
model_RNN.fit(trainX,trainY,epochs=epochs,batch_size=1,verbose=2)

#Evalute model
train_mse=model_RNN.evaluate(trainX,trainY)
test_mse=model_RNN.evaluate(testX,testY)

#Print error
print("Train set MSE =",train_mse)
print("Test set MSE =",test_mse)


Epoch 1/30
826/826 - 2s - 2ms/step - loss: 7.0117e-05
Epoch 2/30
826/826 - 2s - 3ms/step - loss: 6.9179e-05
Epoch 3/30
826/826 - 3s - 4ms/step - loss: 6.6072e-05
Epoch 4/30
826/826 - 4s - 5ms/step - loss: 6.6385e-05
Epoch 5/30
826/826 - 2s - 2ms/step - loss: 6.4761e-05
Epoch 6/30
826/826 - 3s - 3ms/step - loss: 6.5112e-05
Epoch 7/30
826/826 - 3s - 3ms/step - loss: 6.3880e-05
Epoch 8/30
826/826 - 5s - 5ms/step - loss: 6.3367e-05
Epoch 9/30
826/826 - 2s - 2ms/step - loss: 6.3467e-05
Epoch 10/30
826/826 - 3s - 4ms/step - loss: 6.4160e-05
Epoch 11/30
826/826 - 2s - 3ms/step - loss: 6.2961e-05
Epoch 12/30
826/826 - 3s - 3ms/step - loss: 6.3692e-05
Epoch 13/30
826/826 - 2s - 2ms/step - loss: 6.0532e-05
Epoch 14/30
826/826 - 3s - 3ms/step - loss: 5.8078e-05
Epoch 15/30
826/826 - 2s - 2ms/step - loss: 6.2592e-05
Epoch 16/30
826/826 - 2s - 2ms/step - loss: 6.0767e-05
Epoch 17/30
826/826 - 3s - 4ms/step - loss: 6.0394e-05
Epoch 18/30
826/826 - 4s - 5ms/step - loss: 6.0358e-05
Epoch 19/30
826/826

The Call Method for the Attention LAyer

In [18]:
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

RNN Network with Attention Layer

In [19]:
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='mse', optimizer='adam')
    return model

model_attention = create_RNN_with_attention(hidden_units=hidden_units, dense_units=1,
                                  input_shape=(time_steps,1), activation='tanh')
model_attention.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 20, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_3 (SimpleRNN)             │ (None, 20, 2)               │               8 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_1 (attention)              │ (None, 2)                   │              22 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33 (132.00 B)

 Trainable params: 33 (132.00 B)

 Non-trainable params: 0 (0.00 B)

Train and Evalute the Deep Learning Network with attention

In [20]:
model_attention.fit(trainX,trainY,epochs=epochs,batch_size=1,verbose=2)

#Evalute model
train_mse_attn=model_attention.evaluate(trainX,trainY)
test_mse_attn=model_attention.evaluate(testX,testY)

#Print error
print("Train set MSE with attention =",train_mse_attn)
print("Test set MSE with attention =",test_mse_attn)

Epoch 1/30
826/826 - 4s - 4ms/step - loss: 0.0015
Epoch 2/30
826/826 - 3s - 3ms/step - loss: 0.0015
Epoch 3/30
826/826 - 3s - 4ms/step - loss: 0.0015
Epoch 4/30
826/826 - 2s - 3ms/step - loss: 0.0015
Epoch 5/30
826/826 - 3s - 3ms/step - loss: 0.0015
Epoch 6/30
826/826 - 2s - 3ms/step - loss: 0.0015
Epoch 7/30
826/826 - 3s - 3ms/step - loss: 0.0015
Epoch 8/30
826/826 - 3s - 4ms/step - loss: 0.0015
Epoch 9/30
826/826 - 3s - 3ms/step - loss: 0.0015
Epoch 10/30
826/826 - 5s - 6ms/step - loss: 0.0014
Epoch 11/30
826/826 - 5s - 6ms/step - loss: 0.0014
Epoch 12/30
826/826 - 3s - 3ms/step - loss: 0.0014
Epoch 13/30
826/826 - 2s - 3ms/step - loss: 0.0014
Epoch 14/30
826/826 - 2s - 3ms/step - loss: 0.0014
Epoch 15/30
826/826 - 2s - 3ms/step - loss: 0.0014
Epoch 16/30
826/826 - 3s - 3ms/step - loss: 0.0014
Epoch 17/30
826/826 - 4s - 4ms/step - loss: 0.0014
Epoch 18/30
826/826 - 4s - 5ms/step - loss: 0.0013
Epoch 19/30
826/826 - 3s - 3ms/step - loss: 0.0013
Epoch 20/30
826/826 - 2s - 3ms/step - lo

Consolidated Code


In [21]:
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
K = tf.keras.backend
from keras.layers import Input,Dense,SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

In [23]:
#Prepare data
def get_fib_seq(n,scale_data=True):
  #Get the Fibonacci sequence
  seq=np.zeros(n)
  fib_n1=0.0
  fib_n=1.0
  for i in range(n):
    seq[i]=fib_n1+fib_n
    fib_n1=fib_n
    fib_n=seq[i]

  scaler=[]
  if scale_data:
    scaler=MinMaxScaler(feature_range=(0,1))
    seq=np.reshape(seq,(n,1))
    seq=scaler.fit_transform(seq).flatten()

  return seq,scaler

def get_fib_XY(total_fib_numbers,time_steps,train_percent,scale_data=True):
  dat,scaler=get_fib_seq(total_fib_numbers,scale_data)
  Y_ind=np.arange(time_steps,len(dat),1)
  Y=dat[Y_ind]
  rows_x=len(Y)
  X=dat[0:rows_x]
  for i in range(time_steps-1):
    temp=dat[i+1:rows_x+i+1]
    X=np.column_stack((X,temp))
  # random permutation with fixed seed
  rand=np.random.RandomSate(seed=13)
  idx=rand.permutation(rows_x)
  split=int(train_percent*rows_x)
  train_ind=idx[0:split]
  test_ind=idx[split:]
  trainX=X[train_ind]
  trainY=Y[train_ind]
  testX=X[test_ind]
  testY=Y[test_ind]
  trainX=np.reshape(trainX,(len(trainX),time_steps,1))
  testX=np.reshape(testX,(len(testX),time_steps,1))
  return trainX,trainY,testX,testY,scaler

#Set up parameters
time_steps=20
hidden_units=2
epochs=30

#Create a traditional RNN network
def create_RNN(hidden_units,dense_units,input_shape,activation):
  model=Sequential()
  model.add(SimpleRNN(hidden_units,input_shape=input_shape,activation=activation[0]))
  model.add(Dense(units=dense_units,activation=activation[1]))
  model.compile(loss='mse',optimizer='adam')
  return model

  model_RNN=create_RNN(hidden_units=hidden_units,dense_units=1,input_shape=(time_steps,1),activation=['tanh','tanh'])

  #Generate the daatset for the network
  trainX,trainY,testX,testY,scaler=get_fib_XY(1200,time_steps,0.7)
  #train the network
  model_RNN.fit(trainX,trainY,epochs=epochs,batch_size=1,verbose=2)

  #Evalute model
  train_mse=model_RNN>evaluate(trainX,trainY)
  test_mse=model_RNN.evaluate(testX,testY)

  #Print error
  print("Train set MSE = ",train_mse)
  print("Test set MSE =" ,test_mse)

  #Add attention layer to the deep learning network
  class attention(layer):
    def __init__(self,**kwargs):
      super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
      self.W=self.add_weight(name='attention_weight',shape=(input_shape[-1],1),
                             initializer='random_normal',trainable=True)
      self.b=self.add_weight(name='attention_bias',shape=(input_shape[1],1),
                             initializer='zeros',trainable=True)
      super(attention,self).build(input_shape)


    def call(self,x):
      #Alignment scores .Pass them through tanh function
      e=K.tanh(K.dot(x,self.W)+self.b)
      #Remove dimension of size 1
      e=K.squeeze(e,axis=-1)
      #compute the weights
      alpha=K.softmax(e)
      #reshape to tensorflow format
      alpha=K.expand_dims(alpha,axis=-1)
      #compute the context vector
      context=x*alpha
      context=K.sum(context,axis=1)
      return context

def create_RNN_with_attention(hidden_units,dense_units,input_shape,activation):
  x=Input(shape=input_shape)
  RNN_layer=SimpleRNN(hidden_units,return_sequences=True,activation=activation)(x)
  attention_layer=attention()(RNN_layer)
  outputs=Dense(dense_units,trainable=True,activation=activation)(attention_layer)
  model=Model(x,outputs)
  model.compile(loss='mse',optimizer='adam')
  return model

model_attention=create_RNN_with_attention(hidden_units=hidden_units,dense_units=1,
                                           input_shape=(time_steps,1),activation='tanh')


#Create the model with attention ,train and evaluate
model_attention=create_RNN_with_attention(hidden_units=hidden_units,dense_units=1,
                                           input_shape=(time_steps,1),activation='tanh')
model_attention.summary()
model_attention.fit(trainX,trainY,epochs=epochs,batch_size=1,verbose=2)

#Evalute model
train_mse_attn=model_attention.evaluate(trainX,trainY)
test_mse_attn=model_attention.evaluate(testX,testY)

#Print error
print("Train set MSE with attention =",train_mse_attn)
print("Test set MSE with attention =",test_mse_attn)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 20, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_5 (SimpleRNN)             │ (None, 20, 2)               │               8 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_3 (attention)              │ (None, 2)                   │              22 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33 (132.00 B)

 Trainable params: 33 (132.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
826/826 - 4s - 5ms/step - loss: 0.0014
Epoch 2/30
826/826 - 3s - 3ms/step - loss: 0.0014
Epoch 3/30
826/826 - 4s - 5ms/step - loss: 0.0013
Epoch 4/30
826/826 - 2s - 3ms/step - loss: 0.0013
Epoch 5/30
826/826 - 2s - 3ms/step - loss: 0.0013
Epoch 6/30
826/826 - 4s - 4ms/step - loss: 0.0012
Epoch 7/30
826/826 - 2s - 3ms/step - loss: 0.0012
Epoch 8/30
826/826 - 2s - 3ms/step - loss: 0.0011
Epoch 9/30
826/826 - 3s - 3ms/step - loss: 0.0011
Epoch 10/30
826/826 - 3s - 3ms/step - loss: 9.9939e-04
Epoch 11/30
826/826 - 3s - 4ms/step - loss: 9.8775e-04
Epoch 12/30
826/826 - 4s - 5ms/step - loss: 8.9880e-04
Epoch 13/30
826/826 - 3s - 3ms/step - loss: 8.3923e-04
Epoch 14/30
826/826 - 2s - 3ms/step - loss: 7.6871e-04
Epoch 15/30
826/826 - 3s - 3ms/step - loss: 7.2267e-04
Epoch 16/30
826/826 - 3s - 4ms/step - loss: 6.4717e-04
Epoch 17/30
826/826 - 4s - 5ms/step - loss: 5.9041e-04
Epoch 18/30
826/826 - 2s - 3ms/step - loss: 5.2878e-04
Epoch 19/30
826/826 - 3s - 3ms/step - loss: 4.7626e-04
